In [1]:
import pandas as pd
import numpy as np

Changed in version 1.0.0: Now uses pandas.NA as the missing value rather than numpy.nan.

In Working with missing data, we saw that pandas primarily uses NaN to represent missing data. Because NaN is a float, this forces an array of integers with any missing values to become floating point. In some cases, this may not matter much. But if your integer column is, say, an identifier, casting to float can be problematic. Some integers cannot even be represented as floating point numbers.

# Construction
pandas can represent integer data with possibly missing values using arrays.IntegerArray. This is an extension types implemented within pandas.

In [2]:
arr = pd.array([1, 2, None], dtype=pd.Int64Dtype())
arr

<IntegerArray>
[1, 2, <NA>]
Length: 3, dtype: Int64

Or the string alias "Int64" (note the capital "I", to differentiate from NumPy’s 'int64' dtype:

In [3]:
pd.array([1, 2, np.nan], dtype="Int64")

<IntegerArray>
[1, 2, <NA>]
Length: 3, dtype: Int64

All NA-like values are replaced with pandas.NA.

In [4]:
pd.array([1, 2, np.nan, None, pd.NA], dtype="Int64")

<IntegerArray>
[1, 2, <NA>, <NA>, <NA>]
Length: 5, dtype: Int64

This array can be stored in a DataFrame or Series like any NumPy array.

In [5]:
pd.Series(arr)

0       1
1       2
2    <NA>
dtype: Int64

You can also pass the list-like object to the Series constructor with the dtype.

Currently pandas.array() and pandas.Series() use different rules for dtype inference. pandas.array() will infer a nullable- integer dtype

In [6]:
pd.array([1, None])

<IntegerArray>
[1, <NA>]
Length: 2, dtype: Int64

In [7]:
 pd.array([1, 2])

<IntegerArray>
[1, 2]
Length: 2, dtype: Int64

For backwards-compatibility, Series infers these as either integer or float dtype

In [8]:
pd.Series([1, None])

0    1.0
1    NaN
dtype: float64

In [9]:
pd.Series([1, 2])

0    1
1    2
dtype: int64

We recommend explicitly providing the dtype to avoid confusion.

In [10]:
pd.array([1, None], dtype="Int64")

<IntegerArray>
[1, <NA>]
Length: 2, dtype: Int64

In [11]:
pd.Series([1, None], dtype="Int64")

0       1
1    <NA>
dtype: Int64

In the future, we may provide an option for Series to infer a nullable-integer dtype.

# Operations
Operations involving an integer array will behave similar to NumPy arrays. Missing values will be propagated, and the data will be coerced to another dtype if needed.

In [12]:
s = pd.Series([1, 2, None], dtype="Int64")

In [13]:
# arithmetic
s + 1

0       2
1       3
2    <NA>
dtype: Int64

In [14]:
# comparison
s == 1

0     True
1    False
2     <NA>
dtype: boolean

In [15]:
# indexing
s.iloc[1:3]

1       2
2    <NA>
dtype: Int64

In [16]:
# operate with other dtypes
s + s.iloc[1:3].astype("Int8")

0    <NA>
1       4
2    <NA>
dtype: Int64

In [17]:
# coerce when needed
s + 0.01

0    1.01
1    2.01
2    <NA>
dtype: Float64

These dtypes can operate as part of DataFrame.

In [19]:
df = pd.DataFrame({"A": s, "B": [1, 1, 3], "C": list("aab")})
df

,A,B,C
0,1,1,a
1,2,1,a
2,<NA>,3,b


In [20]:
df.dtypes

A     Int64
B     int64
C    object
dtype: object

These dtypes can be merged & reshaped & casted.

In [21]:
pd.concat([df[["A"]], df[["B", "C"]]], axis=1).dtypes

A     Int64
B     int64
C    object
dtype: object

In [22]:
df["A"].astype(float)

0    1.0
1    2.0
2    NaN
Name: A, dtype: float64

Reduction and groupby operations such as ‘sum’ work as well.

In [23]:
df.sum()

A      3
B      5
C    aab
dtype: object

In [24]:
df.groupby("B").A.sum()

B
1    3
3    0
Name: A, dtype: Int64

# Scalar NA Value
arrays.IntegerArray uses pandas.NA as its scalar missing value. Slicing a single element that’s missing will return pandas.NA

In [25]:
a = pd.array([1, None], dtype="Int64")
a[1]

<NA>